## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [ ]:
import pyvista as pv
from collab_splats.wrapper import Splatter

pv.start_xvfb()

## Load the splatter from configuration

Load the dataset configuration from YAML and ensure preprocessing/training steps are complete:

In [ ]:
# Load splatter from YAML config
splatter = Splatter.from_config_file(
    dataset='birds_date-02062024_video-C0043',
    config_dir='configs'
)

# Ensure preprocessing and training are done
# (if already completed, these will skip automatically)
splatter.preprocess()
splatter.extract_features()

In [ ]:
# Create the mesh (config already contains meshing parameters)
splatter.mesh(overwrite=False)

### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [ ]:
splatter.plot_mesh(attribute="RGB")

wslink is not expecting text message:
> No PONG received after 15.0 seconds
wslink is not expecting text message:
> No PONG received after 15.0 seconds
wslink is not expecting text message:
> No PONG received after 15.0 seconds


In [ ]:
similarity = splatter.query_mesh(
    positive_queries=["feeder"], negative_queries=["ground", "leaves", "rocks"], output_fn="query-feeder.ply"
)

Plot similarity maps

In [ ]:
splatter.plot_mesh(attribute=similarity, rgb=False)

#### Semantic clustering

In [ ]:
import numpy as np
import open3d as o3d

from collab_splats.utils.mesh import mesh_clustering

# Load our query mesh
mesh_dir = splatter.config["mesh_info"]["mesh"].parent
query_mesh = mesh_dir / "query-feeder.ply"
mesh = o3d.io.read_triangle_mesh(query_mesh)
similarity = np.asarray(mesh.vertex_colors)[:, 0]

# Cluster based on similarity values
clusters = mesh_clustering(
    mesh=mesh,
    similarity_values=similarity,
    similarity_threshold=0.9,
    spatial_radius=0.01,
)

Select the largest cluster

In [ ]:
# Get sizes of each cluster
cluster_sizes = [x.shape[0] for x in clusters]
cluster_size_idxs = np.argsort(cluster_sizes)[::-1]

# Pick largest cluster
target_cluster = clusters[cluster_size_idxs[0]]
cluster_vertices = np.array(target_cluster)  # Convert to numpy array

# Color the mesh to highlight the cluster
colors = np.zeros((len(mesh.vertices), 3))  # Gray background
colors[cluster_vertices] = [1, 0, 0]  # Red for selected cluster

Copy from original mesh and set properties

In [ ]:
import copy

cluster_mesh = copy.deepcopy(mesh)
cluster_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

o3d.io.write_triangle_mesh(mesh_dir / "query-feeder_top-cluster.ply", cluster_mesh)

Plot the top cluster

In [ ]:
splatter.plot_mesh(colors)